In [23]:
import os
import csv
import numpy as np

In [24]:
main_dir ="/Users/08volt/Desktop/FINAL/output_moa"
old_drifts = ["sea_P(y)","sea_P(y_x)","sea_P(x_y)","sine_P(y)","sine_P(y_x)","sine_P(x_y)"]
new_drifts = ["jitter","disappearing-minority","appearing-minority","appearing-clusters",
             "shapeshift","minority-share","splitting-clusters","clusters-movement","borderline"]
type_drift = ["incremental","sudden","periodic"]
unbalance_lvls = [1,2,3,4]
stats_w = ["Recall_0","Recall_1", "F1_0", "F1_1", "Gmean"]
stats_tot =["time", "memory"]
algs = ["RebalanceStream","CSMOTE","improved_UOB","improved_OOB","WEOB1","WEOB2","ESOS_ELM","UOB","OOB"]

In [25]:
def import_csv(csvfilename):
    data = []
    with open(csvfilename, "r") as scraped:
        reader = csv.reader(scraped, delimiter=',')
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                row_index += 1
                columns = [str(row_index)]+ [row[i] for i in range(len(row))]
                data.append(columns)
        #print(f"{row_index} valid rows")
        
    return data

In [26]:
time_dict = {}

for old in old_drifts:
    path = main_dir + "/" + old[old.index("P"):]   #old = sea_P(y)
    if old not in time_dict:
        time_dict[old] = {}
        
    for ds in os.listdir(path):
        #check if ds in old (directory giusta)
        if old[:3] == ds[:3]:
            path1 = path + "/" + ds
            
            unbalance = 0
            for u in unbalance_lvls:
                if str(u) in ds:
                    unbalance = u
            assert unbalance != 0
                        
            if unbalance not in time_dict[old]:
                time_dict[old][unbalance] = {}
                
            dtype = "sudden"
            if "G" in ds:
                dtype = "incremental"
                
            if dtype not in time_dict[old][unbalance]:
                time_dict[old][unbalance][dtype] = {}

            for alg in os.listdir(path1):
                if "DS_Store" in alg:
                    continue
                
                path2 = path1 + "/" + alg
                if alg == "OSHAT":
                    alg = "CSMOTE"
                times = []
                
                #print(f"{old} {unbalance} {dtype} {alg}")
                #prendo i valori di tutti i 10 esperimenti
                for result in os.listdir(path2):
                    if "err" not in result or "DS_Store" in result:
                        continue
                    
                    data = import_csv(path2+"/"+ result)
                    
#                     for row in data:
#                         if row[1] == "100000.0" or row[1] == "100000":
#                             times.append(float(row[2]))
                        #print(row)
                    try:
                        times.append(float(data[-1][2]))
                    except:
                        print(f"error with {result}")
                
                assert len(times) > 0
                time_dict[old][unbalance][dtype][alg] = {}
                time_dict[old][unbalance][dtype][alg]["Time"] = np.mean(times)
                #print(times)
                #print(np.mean(times))
                    
                    
                
        

In [27]:
time_dict

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'Time': 6.456626299999999},
    'WEOB1': {'Time': 10.0528391},
    'OOB': {'Time': 6.961646},
    'HoeffdingAdaptiveTree': {'Time': 2.5461970999999997},
    'ESOS_ELM': {'Time': 342.4281187911},
    'OzaBag': {'Time': 9.317198},
    'improved_UOB': {'Time': 4.9094462},
    'WEOB2': {'Time': 15.952558000000002},
    'UOB': {'Time': 7.159590199999999},
    'CSMOTE': {'Time': 28.6611699175},
    'RebalanceStream': {'Time': 18.0669322}},
   'incremental': {'improved_OOB': {'Time': 6.3291709},
    'WEOB1': {'Time': 9.5864254},
    'OOB': {'Time': 8.3814722},
    'HoeffdingAdaptiveTree': {'Time': 2.5163532},
    'ESOS_ELM': {'Time': 335.1285003525},
    'OzaBag': {'Time': 9.085131},
    'improved_UOB': {'Time': 4.7329115},
    'WEOB2': {'Time': 15.6431828},
    'UOB': {'Time': 8.6141637},
    'CSMOTE': {'Time': 222.63570002390003},
    'RebalanceStream': {'Time': 14.7059968}}},
  3: {'incremental': {'improved_OOB': {'Time': 9.518834899999998},
  

In [28]:
for new in new_drifts:
    print(path)
    path = main_dir + "/" + new 
    
    if new not in time_dict:
        time_dict[new] = {}
        
    for unbalance in range(1,5):
        if unbalance not in time_dict[new]:
            time_dict[new][unbalance] = {}


        path1 = path + "/" + str(unbalance)

        for alg in os.listdir(path1):
            if "DS_Store" in alg:
                continue
            if "Oza" in alg:
                continue
            
            path2 = path1 + "/" + alg
            
            alg_dtypes = {}
            alg_dtypes["incremental"] = []
            alg_dtypes["sudden"] = []
            alg_dtypes["periodic"] = []
            
            
            print(f"{new} {unbalance} {alg}")
            #prendo i valori di tutti i 10 esperimenti dei 3 dtype
            for result in os.listdir(path2):
                if "err" not in result or "DS_Store" in result:
                    continue
                print(result)
                for i in range(0,10):
                    if str(i) in result:
                        break
                if i > 3 or i < 1:
                    continue
                

                data = import_csv(path2+"/"+ result)
    
                try:
                    alg_dtypes[result[:result.index("_")]].append(float(data[-1][2]))
                except:
                    print(f"error with {result}")
            for k,i in alg_dtypes.items():      
                if len(i) <= 0:
                    print(f"{k} -> NODATA")
                    continue
                
                if k not in time_dict[new][unbalance]:
                    time_dict[new][unbalance][k] = {}
                assert alg not in time_dict[new][unbalance][k]
                time_dict[new][unbalance][k][alg] = {}
                time_dict[new][unbalance][k][alg]["Time"] = np.mean(i)
                print(np.mean(i))

/Users/08volt/Desktop/FINAL/output_moa/P(x_y)
jitter 1 improved_OOB
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
8.130513
19.544876666666664
11.376499
jitter 1 WEOB1
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incre

sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
16.130629
17.017264333333333
16.634131999999997
jitter 3 CSMOTE
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
494.3833683333334
506.90088466666674
507.3749383333333
jitter 3 OOB
incremental_8_err.csv
sudden_9_err

periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
7.39046
6.309744333333334
10.526292
disappearing-minority 1 HoeffdingAdaptiveTree
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
2.366616666666667
2.5436183333333333
2.604759
disappearing-minority 1 ESOS_ELM
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incre

periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
7.729610000000001
6.544156666666666
9.139692
disappearing-minority 3 WEOB1
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
11.147993333333332
10.135262333

sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
6.577613333333333
6.121123999999999
7.917607
disappearing-minority 4 RebalanceStream
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incre

...

sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
7.919328666666666
7.443066999999999
7.689191
splitting-clusters 4 WEOB1
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.

incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
338.4524164333333
314.301902838
323.5500112926666
clusters-movement 1 improved_UOB
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
su

incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
8.969664333333332
8.441657666666666
8.655790333333334
clusters-movement 3 WEOB1
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
14.030639666666666
13.788099333333333
14.614078999999998
clusters-movement 3 CSMOTE
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv

incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
2.456581
2.4238416666666667
2.7386086666666665
borderline 1 ESOS_ELM
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.c

periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
7.212445333333334
7.084851333333333
7.213428666666666
borderline 3 WEOB2
incremental_8_err.csv
sudden_9_err.csv
periodic_4_err.csv
incremental_1_err.csv
sudden_8_err.csv
periodic_5_err.csv
incremental_9_err.csv
sudden_1_err.csv
incremental_2_err.csv
periodic_7_err.csv
sudden_3_err.csv
periodic_10_err.csv
sudden_2_err.csv
incremental_3_err.csv
periodic_6_err.csv
incremental_6_err.csv
periodic_3_err.csv
sudden_7_err.csv
sudden_6_err.csv
incremental_10_err.csv
sudden_10_err.csv
incremental_7_err.csv
periodic_2_err.csv
periodic_9_err.csv
sudden_4_err.csv
incremental_5_err.csv
periodic_1_err.csv
incremental_4_err.csv
periodic_8_err.csv
sudden_5_err.csv
16.79928333333333
16.428877666666

In [29]:
time_dict

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'Time': 6.456626299999999},
    'WEOB1': {'Time': 10.0528391},
    'OOB': {'Time': 6.961646},
    'HoeffdingAdaptiveTree': {'Time': 2.5461970999999997},
    'ESOS_ELM': {'Time': 342.4281187911},
    'OzaBag': {'Time': 9.317198},
    'improved_UOB': {'Time': 4.9094462},
    'WEOB2': {'Time': 15.952558000000002},
    'UOB': {'Time': 7.159590199999999},
    'CSMOTE': {'Time': 28.6611699175},
    'RebalanceStream': {'Time': 18.0669322}},
   'incremental': {'improved_OOB': {'Time': 6.3291709},
    'WEOB1': {'Time': 9.5864254},
    'OOB': {'Time': 8.3814722},
    'HoeffdingAdaptiveTree': {'Time': 2.5163532},
    'ESOS_ELM': {'Time': 335.1285003525},
    'OzaBag': {'Time': 9.085131},
    'improved_UOB': {'Time': 4.7329115},
    'WEOB2': {'Time': 15.6431828},
    'UOB': {'Time': 8.6141637},
    'CSMOTE': {'Time': 222.63570002390003},
    'RebalanceStream': {'Time': 14.7059968}}},
  3: {'incremental': {'improved_OOB': {'Time': 9.518834899999998},
  

In [30]:
time_dict["minority-share"][4]

{'incremental': {'improved_OOB': {'Time': 5.992992999999999},
  'WEOB1': {'Time': 9.806878},
  'CSMOTE': {'Time': 139.45881},
  'OOB': {'Time': 7.320729666666666},
  'HoeffdingAdaptiveTree': {'Time': 2.3474049999999997},
  'ESOS_ELM': {'Time': 260.726893608},
  'improved_UOB': {'Time': 4.3394070000000005},
  'WEOB2': {'Time': 9.632801333333333},
  'UOB': {'Time': 5.481831},
  'RebalanceStream': {'Time': 14.524449666666667}},
 'sudden': {'improved_OOB': {'Time': 5.656348333333334},
  'WEOB1': {'Time': 9.407685},
  'CSMOTE': {'Time': 156.298239},
  'OOB': {'Time': 6.645066},
  'HoeffdingAdaptiveTree': {'Time': 2.178487},
  'ESOS_ELM': {'Time': 247.562412298},
  'improved_UOB': {'Time': 4.344391000000001},
  'WEOB2': {'Time': 9.061098000000001},
  'UOB': {'Time': 5.499252666666666},
  'RebalanceStream': {'Time': 17.181981}},
 'periodic': {'improved_OOB': {'Time': 4.255312666666666},
  'WEOB1': {'Time': 6.849085333333332},
  'CSMOTE': {'Time': 22.082130333333335},
  'OOB': {'Time': 5.43745

In [31]:
#WEOB2 = #WEOB1

#fermati a exp 3

In [32]:
# time_dict["minority-share"][4]["incremental"]["RebalanceStream"] = time_dict["minority-share"][3]["incremental"]["RebalanceStream"].copy()
# time_dict["minority-share"][4]["sudden"]["RebalanceStream"] = time_dict["minority-share"][3]["sudden"]["RebalanceStream"].copy()
# time_dict["minority-share"][4]["periodic"]["RebalanceStream"] = time_dict["minority-share"][3]["periodic"]["RebalanceStream"].copy()

# time_dict["minority-share"][4]["incremental"]["UOB"] = time_dict["minority-share"][3]["incremental"]["UOB"].copy()
# time_dict["minority-share"][4]["sudden"]["UOB"] = time_dict["minority-share"][3]["sudden"]["UOB"].copy()
# time_dict["minority-share"][4]["periodic"]["UOB"] = time_dict["minority-share"][3]["periodic"]["UOB"].copy()

# time_dict["minority-share"][4]["incremental"]["WEOB2"] = time_dict["minority-share"][3]["incremental"]["WEOB2"].copy()
# time_dict["minority-share"][4]["sudden"]["WEOB2"] = time_dict["minority-share"][3]["sudden"]["WEOB2"].copy()
# time_dict["minority-share"][4]["periodic"]["WEOB2"] = time_dict["minority-share"][3]["periodic"]["WEOB2"].copy()

# time_dict["minority-share"][4]["incremental"]["OOB"] = time_dict["minority-share"][3]["incremental"]["OOB"].copy()
# time_dict["minority-share"][4]["sudden"]["OOB"] = time_dict["minority-share"][3]["sudden"]["OOB"].copy()
# time_dict["minority-share"][4]["periodic"]["OOB"] = time_dict["minority-share"][3]["periodic"]["OOB"].copy()

#time_dict["minority-share"][3]["sudden"]["CSMOTE"] = time_dict["minority-share"][2]["sudden"]["CSMOTE"].copy()

# time_dict["minority-share"][4]["incremental"]["CSMOTE"] = time_dict["minority-share"][3]["incremental"]["CSMOTE"].copy()
# time_dict["minority-share"][4]["sudden"]["CSMOTE"] = time_dict["minority-share"][3]["sudden"]["CSMOTE"].copy()
# time_dict["minority-share"][4]["periodic"]["CSMOTE"] = time_dict["minority-share"][3]["periodic"]["CSMOTE"].copy()



In [33]:
final_dict = time_dict.copy()

In [34]:
final_dict

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'Time': 6.456626299999999},
    'WEOB1': {'Time': 10.0528391},
    'OOB': {'Time': 6.961646},
    'HoeffdingAdaptiveTree': {'Time': 2.5461970999999997},
    'ESOS_ELM': {'Time': 342.4281187911},
    'OzaBag': {'Time': 9.317198},
    'improved_UOB': {'Time': 4.9094462},
    'WEOB2': {'Time': 15.952558000000002},
    'UOB': {'Time': 7.159590199999999},
    'CSMOTE': {'Time': 28.6611699175},
    'RebalanceStream': {'Time': 18.0669322}},
   'incremental': {'improved_OOB': {'Time': 6.3291709},
    'WEOB1': {'Time': 9.5864254},
    'OOB': {'Time': 8.3814722},
    'HoeffdingAdaptiveTree': {'Time': 2.5163532},
    'ESOS_ELM': {'Time': 335.1285003525},
    'OzaBag': {'Time': 9.085131},
    'improved_UOB': {'Time': 4.7329115},
    'WEOB2': {'Time': 15.6431828},
    'UOB': {'Time': 8.6141637},
    'CSMOTE': {'Time': 222.63570002390003},
    'RebalanceStream': {'Time': 14.7059968}}},
  3: {'incremental': {'improved_OOB': {'Time': 9.518834899999998},
  

In [51]:
#OLD DRIFTS
#csv_file = open('mem.csv', 'w')
# writer = csv.writer(csv_file)
# writer.writerow(["drift","unbalance","type","alg","mem"])
main_dir ="/Users/08volt/Desktop/StremingML/TEST/Combined"

for d in os.listdir(main_dir): #PER OGNI CARTELLA DRIFT
    if d[0] != "(":
        continue
    print(d)
    for file in os.listdir(main_dir+ "/" + d):
        print(file)
        ds = file[:file.index("_")]
        unbalance = int(file[file.index("_")+1])
        dtype = "sudden"
        if "G" == ds[-1]:
            dtype = "incremental"
            ds = ds[:-1]
        data = import_csv(main_dir+ "/" + d + "/" + file)[1:]
        
        drift = ""
        for old in old_drifts:
            if d in old and ds in old:
                drift = old
                break
        assert drift != ""
        
        for row in data:
            alg = row[2]
            if alg == "WEOB2":
                print(alg)
                continue
            
            value = float(row[3])
            final_dict[old][unbalance][dtype][alg]["Mem"] = value
            #writer.writerow([drift,unbalance,dtype,alg,value])

(y_x)
sea_2_8.csv
WEOB2
sineG_4_6.csv
WEOB2
seaG_1_9.csv
WEOB2
sine_3_7.csv
WEOB2
sea_1_9.csv
WEOB2
sine_4_6.csv
WEOB2
seaG_2_8.csv
WEOB2
sineG_3_7.csv
WEOB2
sineG_1_9.csv
WEOB2
sine_2_8.csv
WEOB2
seaG_4_6.csv
WEOB2
sea_3_7.csv
WEOB2
sea_4_6.csv
WEOB2
sineG_2_8.csv
WEOB2
seaG_3_7.csv
WEOB2
sine_1_9.csv
WEOB2
(x_y)
sea_2_8.csv
WEOB2
sineG_4_6.csv
WEOB2
seaG_1_9.csv
WEOB2
sine_3_7.csv
WEOB2
sea_1_9.csv
WEOB2
sine_4_6.csv
WEOB2
seaG_2_8.csv
WEOB2
sineG_3_7.csv
WEOB2
sineG_1_9.csv
WEOB2
sine_2_8.csv
WEOB2
seaG_4_6.csv
WEOB2
sea_3_7.csv
WEOB2
sea_4_6.csv
WEOB2
sineG_2_8.csv
WEOB2
seaG_3_7.csv
WEOB2
sine_1_9.csv
WEOB2
(y)
sea_2_8.csv
WEOB2
sineG_4_6.csv
WEOB2
seaG_1_9.csv
WEOB2
sine_3_7.csv
WEOB2
sea_1_9.csv
WEOB2
sine_4_6.csv
WEOB2
seaG_2_8.csv
WEOB2
sineG_3_7.csv
WEOB2
sineG_1_9.csv
WEOB2
sine_2_8.csv
WEOB2
seaG_4_6.csv
WEOB2
sea_3_7.csv
WEOB2
sea_4_6.csv
WEOB2
sineG_2_8.csv
WEOB2
seaG_3_7.csv
WEOB2
sine_1_9.csv
WEOB2


In [42]:
for drift in final_dict:
    for im in final_dict[drift]:
        for s in final_dict[drift][im]:
            for alg in final_dict[drift][im][s]:
                if "WEOB1" in alg:
                    print(f"{drift} {im} {s}")
                    if "Mem" in final_dict[drift][im][s]["WEOB1"]:
                        final_dict[drift][im][s]["WEOB2"]["Mem"] = final_dict[drift][im][s]["WEOB1"]["Mem"]
                    
                    else:
                        print("PORCODIO")

sea_P(y) 1 sudden
sea_P(y) 1 incremental
sea_P(y) 3 incremental
sea_P(y) 3 sudden
sea_P(y) 2 sudden
sea_P(y) 2 incremental
sea_P(y) 4 incremental
sea_P(y) 4 sudden
sea_P(y_x) 1 sudden
sea_P(y_x) 1 incremental
sea_P(y_x) 3 incremental
sea_P(y_x) 3 sudden
sea_P(y_x) 2 sudden
sea_P(y_x) 2 incremental
sea_P(y_x) 4 incremental
sea_P(y_x) 4 sudden
sea_P(x_y) 1 sudden
sea_P(x_y) 1 incremental
sea_P(x_y) 3 incremental
sea_P(x_y) 3 sudden
sea_P(x_y) 2 sudden
sea_P(x_y) 2 incremental
sea_P(x_y) 4 incremental
sea_P(x_y) 4 sudden
sine_P(y) 4 incremental
sine_P(y) 4 sudden
sine_P(y) 3 sudden
sine_P(y) 3 incremental
sine_P(y) 1 sudden
sine_P(y) 1 incremental
sine_P(y) 2 incremental
sine_P(y) 2 sudden
sine_P(y_x) 4 incremental
sine_P(y_x) 4 sudden
sine_P(y_x) 3 sudden
sine_P(y_x) 3 incremental
sine_P(y_x) 1 sudden
sine_P(y_x) 1 incremental
sine_P(y_x) 2 incremental
sine_P(y_x) 2 sudden
sine_P(x_y) 4 incremental
sine_P(x_y) 4 sudden
sine_P(x_y) 3 sudden
sine_P(x_y) 3 incremental
sine_P(x_y) 1 sudden
s

In [43]:
final_dict

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'Time': 6.456626299999999,
     'Mem': 540098969.6},
    'WEOB1': {'Time': 10.0528391, 'Mem': 505999769.6},
    'OOB': {'Time': 6.961646, 'Mem': 608969932.8},
    'HoeffdingAdaptiveTree': {'Time': 2.5461970999999997},
    'ESOS_ELM': {'Time': 342.4281187911, 'Mem': 7187554304.0},
    'OzaBag': {'Time': 9.317198},
    'improved_UOB': {'Time': 4.9094462, 'Mem': 402339840.0},
    'WEOB2': {'Time': 15.952558000000002, 'Mem': 505999769.6},
    'UOB': {'Time': 7.159590199999999, 'Mem': 616685568.0},
    'CSMOTE': {'Time': 28.6611699175},
    'RebalanceStream': {'Time': 18.0669322, 'Mem': 4417779302.4}},
   'incremental': {'improved_OOB': {'Time': 6.3291709, 'Mem': 543391334.4},
    'WEOB1': {'Time': 9.5864254, 'Mem': 562321817.6},
    'OOB': {'Time': 8.3814722, 'Mem': 546195865.6},
    'HoeffdingAdaptiveTree': {'Time': 2.5163532},
    'ESOS_ELM': {'Time': 335.1285003525, 'Mem': 5979182545.454545},
    'OzaBag': {'Time': 9.085131},
    'improved_U

In [44]:
for drift in os.listdir(main_dir): #PER OGNI CARTELLA DRIFT
    if drift[0] == "(" or os.path.isfile(main_dir + "/" + drift):
        continue
    cnt = 0
    for file in os.listdir(main_dir+ "/" + drift):
        cnt += 1
        dtype = file[:file.index("_")]
        unbalance = int(file[file.index("_")+1])
        
        data = import_csv(main_dir+ "/" + drift + "/" + file)[1:]
        
        cnt_rows = 0
        for row in data:
            alg = row[2]
            value = float(row[3])
            final_dict[drift][unbalance][dtype][alg]["Mem"] = value
            cnt_rows += 1
            
            #writer.writerow([drift,unbalance,dtype,alg,value])
    print(f"{drift} has {cnt} files ")

minority-share has 12 files 
appearing-minority has 12 files 
appearing-clusters has 12 files 
clusters-movement has 12 files 
splitting-clusters has 12 files 
shapeshift has 12 files 
disappearing-minority has 12 files 
borderline has 12 files 
jitter has 12 files 


In [45]:
def mean_of(_dict ):
    values_0 = []
    values_1 = []
    for alg, values in _dict.items():
        if "Time" in values and "Mem" in values:
            values_0.append(values["Time"])
            values_1.append(values["Mem"])
    return np.mean(values_0),np.mean(values_1)

In [56]:
with open('timeXmemCOMPLETE.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["drift","unbalance","type","alg","Time", "Mem"])#,"Time_mean", "Mem_mean","Time_%", "Mem_%"])
    for drift, dicts in final_dict.items():
        for unbalance, dtypes in dicts.items():
            for dtype, algs in dtypes.items():
                
                time, mem = mean_of(algs)
                if algs["WEOB1"]["Mem"] != algs["WEOB2"]["Mem"]:
                    algs["WEOB2"]["Mem"] = algs["WEOB1"]["Mem"]
                print(algs)
                for alg, values in algs.items():
                    #print([drift,unbalance,dtype,alg])
                    #print(values)
                    if "Mem" in values: 
                        print(alg)
                        writer.writerow([drift,unbalance,dtype,alg,values["Time"],values["Mem"]])#,values["Time"]-time,values["Mem"]-mem,values["Time"]/time - 1,values["Mem"]/mem - 1])

{'improved_OOB': {'Time': 6.456626299999999, 'Mem': 540098969.6}, 'WEOB1': {'Time': 10.0528391, 'Mem': 505999769.6}, 'OOB': {'Time': 6.961646, 'Mem': 608969932.8}, 'HoeffdingAdaptiveTree': {'Time': 2.5461970999999997}, 'ESOS_ELM': {'Time': 342.4281187911, 'Mem': 7187554304.0}, 'OzaBag': {'Time': 9.317198}, 'improved_UOB': {'Time': 4.9094462, 'Mem': 402339840.0}, 'WEOB2': {'Time': 15.952558000000002, 'Mem': 505999769.6}, 'UOB': {'Time': 7.159590199999999, 'Mem': 616685568.0}, 'CSMOTE': {'Time': 28.6611699175}, 'RebalanceStream': {'Time': 18.0669322, 'Mem': 4417779302.4}}
improved_OOB
WEOB1
OOB
ESOS_ELM
improved_UOB
WEOB2
UOB
RebalanceStream
{'improved_OOB': {'Time': 6.3291709, 'Mem': 543391334.4}, 'WEOB1': {'Time': 9.5864254, 'Mem': 562321817.6}, 'OOB': {'Time': 8.3814722, 'Mem': 546195865.6}, 'HoeffdingAdaptiveTree': {'Time': 2.5163532}, 'ESOS_ELM': {'Time': 335.1285003525, 'Mem': 5979182545.454545}, 'OzaBag': {'Time': 9.085131}, 'improved_UOB': {'Time': 4.7329115, 'Mem': 343708392.727

WEOB1
CSMOTE
OOB
ESOS_ELM
improved_UOB
WEOB2
UOB
RebalanceStream
{'improved_OOB': {'Time': 87.88686333333334, 'Mem': 444110506.6666667}, 'WEOB1': {'Time': 77.59317833333334, 'Mem': 489431859.2}, 'CSMOTE': {'Time': 221.693798, 'Mem': 1103911321.6}, 'OOB': {'Time': 84.58872966666667, 'Mem': 525477478.4}, 'HoeffdingAdaptiveTree': {'Time': 2.7386086666666665}, 'ESOS_ELM': {'Time': 282.673209543, 'Mem': 6007907123.2}, 'improved_UOB': {'Time': 71.46883466666667, 'Mem': 279224320.0}, 'WEOB2': {'Time': 86.92399733333333, 'Mem': 489431859.2}, 'UOB': {'Time': 76.33222966666666, 'Mem': 307693568.0}, 'RebalanceStream': {'Time': 62.38706666666667, 'Mem': 2315972198.4}}
improved_OOB
WEOB1
CSMOTE
OOB
ESOS_ELM
improved_UOB
WEOB2
UOB
RebalanceStream
{'improved_OOB': {'Time': 9.714632666666667, 'Mem': 505981610.6666667}, 'WEOB1': {'Time': 14.680392, 'Mem': 619639603.2}, 'CSMOTE': {'Time': 352.3911106666667, 'Mem': 891267891.2}, 'OOB': {'Time': 11.064905333333334, 'Mem': 479988642.90909094}, 'HoeffdingAd

# REALS

In [41]:
mem_dict = {}
mem_dict["KDDCup"] = {}
mem_dict["PAKDD"] = {}
mem_dict["Elec"] = {}

for i in mem_dict.keys():
    mem = import_csv(f"Combined/{i}.csv")[1:]
    for line in mem:
        print(line)
        mem_dict[i][line[2]] = float(line[3])
        


FileNotFoundError: [Errno 2] No such file or directory: 'Combined/KDDCup.csv'

In [51]:
time_dict = {}
time_dict["KDDCup"] = {}
time_dict["PAKDD"] = {}
time_dict["Elec"] = {}

for i in time_dict.keys():
    path = f"Results_union/Real/{i}"
    for alg in os.listdir(path):
        times = []
        if "DS_Store" in alg or "ipynb" in alg:
            continue
        for exp in os.listdir(path + "/" + alg): 
            if "err" not in exp or "DS_Store" in exp:
                continue
                
            data = import_csv(path + "/" + alg + "/"+ exp)
            try:
                times.append(float(data[-1][2]))
            except:
                print(f"error with {result}")
        print(alg)
        assert len(times) > 0
                
        time_dict[i][alg] = np.mean(times)
                

improved_OOB
WEOB1
OOB
ESOS_ELM
improved_UOB
WEOB2
UOB
OSHAT
RebalanceStream
improved_OOB
WEOB1
OOB
ESOS_ELM
improved_UOB
WEOB2
UOB
OSHAT
RebalanceStream
improved_OOB
WEOB1
OOB
ESOS_ELM
improved_UOB
WEOB2
UOB
OSHAT
RebalanceStream


In [53]:
final_dict = {}
for d in time_dict.keys():
    final_dict[d] = {}
    
    for alg in time_dict[d].keys():
        if 'OSHAT' == alg:
            continue
        
        final_dict[d][alg] = {}
        final_dict[d][alg]["Mem"] = mem_dict[d][alg]
        final_dict[d][alg]["Time"] = time_dict[d][alg] 
final_dict    

KeyError: 'RebalanceStream'

In [24]:
with open('MEAN_timeXmem_REAL.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["ds","alg","Time", "Mem"])
    for ds, dicts in final_dict.items():
        time, mem = mean_of(dicts)
        for alg, values in dicts.items():
            writer.writerow([ds,alg,values["Time"] - time,values["Mem"] - mem])